Important info for this to work you must change directory path to your directory in which you cloned the repo. 

In [1]:
directory_path = "C:\\Users\\Crist\\Towncharts\\"

# Merging some data

The datasets being merged only applies to california but if you can find some other datasets that have the name of the cities as a column it should not be too difficult since python and pandas provide convenient ways to work with [text](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html). 

In this notebook i will be merging the `latitude` and `longitude` variables from the kaggle dataset cal_cities_lat_long.csv. A `population` variable for each city after scraping it from towncharts.com demographics web page for each city. And finally, `area_total_km2` variable from Jake Vanderplas hosted california_cities.csv dat. Making it 4 new variable being merged funny enough i notice these last dataset had lat and long

In [2]:
import pandas as pd
import numpy as np

In [3]:

cities_data=pd.read_csv(directory_path+"California_Housing_Project\\datasets\\cal_cities_lat_long.csv")

In [4]:
housing = pd.read_csv(directory_path+"California_Housing_Project\\datasets\\scraped_data.csv",index_col=0)

In [5]:
cities_data.loc[cities_data.Name.str.contains('San Fran')] [['Name']]

,Name
351,San Francisco
400,South San Francisco


In [6]:
housing[housing.city_name.str.contains('San Francisco')][['city_name']]

,city_name
345,San Francisco
394,South San Francisco city-CA


Merging the `latitude` and `longitude`

The following lines of code will create a new city name column in our housing DataFrame that will be used to merge the appropriate latitude and longtitude to the correct row of the housing DataFrame. **The kaggle dataset being merged has the name formatted as ***Alemeda*** and our scraped data has the city formatted as ***Alameda city-CA*** and the ***

In [7]:
mytext = list(housing.city_name.copy())
housing['city'] = [(i.split('city',1)[0]).rstrip()  for i in mytext]

In [8]:
california_housing = housing.merge(cities_data, left_on='city',right_on='Name')

In [9]:
california_housing = california_housing.drop(columns = ['city_name','Name'])

Merging the `population` variable

In [10]:
population_data = pd.read_csv(directory_path+"California_Housing_Project\\datasets\\population_data.csv",index_col=0)

In [11]:
population_data['city'] = population_data['city'].str.replace("-"," ")

In [12]:
california_housing = california_housing.merge(population_data, left_on='city',right_on='city')

Merging `area_total_sq_mi`

In [13]:
area_data = (pd.read_csv('https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/california_cities.csv',index_col=0)).loc[:,['city','area_total_sq_mi']]

The dataset from jake vanderplas has city name with no spaces
so a quick replace should do the job so we are able to merge. Also some cities have missing `area_total_sq_mi`

In [14]:
california_housing['city'] = california_housing['city'].str.replace(" ","")

In [15]:
california_housing = california_housing.merge(area_data,left_on='city',right_on='city',how='left')

And Finally we want to ensure that our column to be used for prediction contains no missing values.

In [16]:
california_housing[pd.isna(california_housing['med_rental_rate'])]

,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,med_hval_aspercentof_medearn,...,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999,city,Latitude,Longitude,population,area_total_sq_mi
165,584.0,345.8,-6.7,3.6,96.4,2000001.0,NaN,4001.0,26.0,1474.0,...,0.0,NaN,NaN,0.684,NaN,HiddenHills,34.160283,-118.652311,1676.0,1.689


In [17]:
california_housing = california_housing[pd.notnull(california_housing['med_rental_rate'])].reset_index(drop=True)

In [18]:
california_housing.to_csv(directory_path+'California_Housing_Project\\datasets\\california_housing.csv')

#  The End